In [1]:
import pandas as pd
import numpy as np
import xarray
import matplotlib.pyplot as plt

In [16]:
#Directory path for plot output
plot_output_DIR = ''

#Data directory paths
reanalysis_DIR = ''
obs_DIR = ''

In [14]:
def roseWheelCP(dataSPD, dataDIR, num=16):
    beaufortlevs1 = np.array([0.0, 5.5, 8., 99.])
    Ncond = (dataDIR >= 348.75) | (dataDIR < 11.25)
    wp = np.pi / 24

    # Exclude calm conditions
    dataDIR = dataDIR[dataSPD > 1.5]
    dataSPD = dataSPD[dataSPD > 1.5]

    # Start with northerlies, crossing direction origin
    sN = dataSPD[Ncond]
    xN = dataDIR[Ncond]

    percSPD = np.zeros([16, 3])
    angs = np.array([np.radians(360) - wp])
    for k in range(3):
        speedcond = (sN > beaufortlevs1[k]) & (sN <= beaufortlevs1[k + 1])
        percSPD[0, k] = (100.0 * xN[speedcond].size) / dataDIR.size
        
    # Now the 15 other direction bins
    for j in range(15):
        dcond = (dataDIR >= ((j + 1) * 22.5) - 11.25) & (dataDIR < ((j + 1) * 22.5) + 11.25)
        xD = dataDIR[dcond]
        sD = dataSPD[dcond]
        angs = np.append(angs, np.radians((j + 1) * 22.5) - wp)
        for k in range(3):
            speedcond = (sD > beaufortlevs1[k]) & (sD <= beaufortlevs1[k + 1])
            percSPD[j + 1, k] = (100.0 * xD[speedcond].size) / dataDIR.size
    return percSPD, angs

def roseplotCP(dataSPD, dataDIR, width,loc,dataset='',title='',limit=14,pthplots='',num=16):
    
    percs,angs = roseWheelCP(dataSPD,dataDIR,num=num)
    fig = plt.figure(figsize=(5,5))
    ax1 = plt.subplot(111, projection='polar')
    ax1.set_theta_zero_location('N')
    ax1.set_theta_direction(-1)
    ax1.set_rlim(0,limit)
    plt.title('%s'%(title),fontsize=15.,fontweight='bold')
    btm=0
    labs = ['1.5 - 5.5 m/s','5.5 - 8.0 m/s','8 m/s +']
    c = ['g','gold','r']
    for i1 in range(3):
        bars1 = ax1.bar(angs+(np.pi/24.), percs[:,i1], width=width, color=c[i1], bottom=btm,label=labs[i1],edgecolor='k')
        btm = np.sum(percs[:,:i1+1],axis=1)
    ax1.grid(linestyle='dotted')
    lgd1 = ax1.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,fontsize=12) #IF you want to include the legend
    plt.tight_layout()
    fig.savefig(plot_output_DIR+loc+'_Rose_'+dataset,bbox_extra_artists=(lgd1,), bbox_inches='tight',dpi=200)
    plt.close(fig)

In [18]:
stn_list=['Belmullet','Birr','Cork_Airport','Claremorris','Clones','Dublin_Airport','Kilkenny','Malin_Head','Mullingar','Rosslare','Shannon_Airport','Valentia_Observatory']


for STN in stn_list:
    print(STN)
    
    #Read in reanalysis and obs data
    m2di = xarray.open_dataset(reanalysis_DIR+STN+'_Merra2Wdir_8207.nc').sel(lat=red_loc.loc[STN].Latitude,lon=red_loc.loc[STN].Longitude,method='nearest').wdir.to_dataframe()['wdir'].rename('MR2 wdir')
    m2ds  = xarray.open_dataset(reanalysis_DIR+STN+'_Merra2Wspd_8207.nc').sel(lat=red_loc.loc[STN].Latitude,lon=red_loc.loc[STN].Longitude,method='nearest').wspd.to_dataframe()['wspd'].rename('MR2 wspd')
    
    edi = xarray.open_dataset(reanalysis_DIR+STN+'_eraWdir_8207.nc').sel(latitude=red_loc.loc[STN].Latitude,longitude=red_loc.loc[STN].Longitude,method='nearest').wdir.to_dataframe()['wdir'].rename('ERA wdir')
    eds = xarray.open_dataset(reanalysis_DIR+STN+'_eraWspd_8207.nc').sel(latitude=red_loc.loc[STN].Latitude,longitude=red_loc.loc[STN].Longitude,method='nearest').wspd.to_dataframe()['wspd'].rename('ERA wspd')

    mdi = pd.read_csv(reanalysis_DIR+'meraWDIR_'+STN+'.csv',index_col=0,parse_dates=True,names=['wdir'])
    mds = pd.read_csv(reanalysis_DIR+'meraWSPD_'+STN+'.csv',index_col=0,parse_dates=True,names=['wspd'])
    
    odi = pd.read_csv(obs_DIR+STN+'_WDIR_obs.csv',index_col=0,parse_dates=True)
    ods = pd.read_csv(obs_DIR+STN+'_WDSP_obs.csv',index_col=0,parse_dates=True,names=['wspd'])


    #Run roseplot functions to produce wind roses
    roseplotCP(m2ds,m2di,np.pi/12,dataset='MERRA2',title=STN.replace('_',' ')+': MERRA2\n',limit=18,loc=STN)
    roseplotCP(eds,edi,np.pi/12,dataset='ERA',title=STN.replace('_',' ')+': ERA\n',limit=18,loc=STN)
    roseplotCP(ods['wspd'],odi['Obs wdir'],np.pi/12,dataset='Obs',title=STN.replace('_',' ')+': Obs\n',limit=18,loc=STN)
    roseplotCP(mds['wspd'],mdi['wdir'],np.pi/12,dataset='MERA',title=STN.replace('_',' ')+': MERA\n',limit=18,loc=STN)

Belmullet
Birr
Clones
Dublin_Airport
Kilkenny
Malin_head
Valentia_Observatory
